In [2]:
import requests
import json
import pickle
import pandas as pd

In [2]:
#First request, API key removed for safety purposes
request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY")
request.status_code

200

In [31]:
#Converting the variable into a dictionary
data_dict=json.loads(request.text)

In [ ]:
#Inspecting the data
data_dict['results']
print(len(data_dict['results'])) #Only 20 restaurants => need to send additional requests for subsequent pages

In [43]:
#Preparing variables for additional requests
next_page_token=json.loads(request.text)['next_page_token']
restaurants=[]
restaurants.extend(json.loads(request.text)['results'])

In [ ]:
#Sending additional requests
i=0
while next_page_token:
    new_request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY&pageToken={next_page_token}")
    if new_request.status_code==200:
        restaurants.extend(json.loads(new_request.text)['results'])
        next_page_token=json.loads(new_request.text)['next_page_token']
        i += 1
    else:
        print(i)
        print(new_request.status_code)
        break
#Note: the loop was broken since I have reached the free quota
#Note 2: API key removed for safety purposes

In [75]:
#Filtering the data
#Empty list
restaurants_filtered=[]
#Keys
keys_new_dict=['name','address','opening_hours','place_id','price_level','rating','no_of_reviews','location']
keys_old_dict=['name','formatted_address','opening_hours','place_id','price_level','rating','user_ratings_total','geometry']
#Looping through restaurants to filter the data
for restaurant in restaurants:
    dict={}
    for key_new, key_old in zip(keys_new_dict, keys_old_dict):
        if key_new=='location': #Location is a dict within adict
            try:
                dict[key_new]=restaurant[key_old][key_new]
            except KeyError:
                dict[key_new]=None
        else: #Rest of the keys
            try:
                dict[key_new]=restaurant[key_old]
            except KeyError:
                dict[key_new]=None
    restaurants_filtered.append(dict)

In [78]:
#Saving the list for future use
with open('restaurants_google_maps_places_API.pkl', 'wb') as data_file:
    pickle.dump(restaurants_filtered, data_file)

In [7]:
#Code to read the file
with open('restaurants_google_maps_places_API.pkl', 'rb') as data_file:
    restaurants_filtered=pickle.load(data_file)

In [10]:
restaurants_filtered

[{'name': 'Restaurant Zvonice',
  'address': 'Jindřišská 33, 110 00 věž-Nové Město, Czechia',
  'place_id': 'ChIJmbWmIO2UC0cR_Gua9yvxWaw',
  'price_level': 3,
  'rating': 4.5,
  'no_of_reviews': 673,
  'location': (50.0851891, 14.4301079)},
 {'name': 'La Finestra in Cucina',
  'address': 'Platnéřská 90/13, 110 00 Staré Město, Czechia',
  'place_id': 'ChIJGxmor-iUC0cRdTmViA87nzs',
  'price_level': 3,
  'rating': 4.6,
  'no_of_reviews': 988,
  'location': (50.08735129999999, 14.4162553)},
 {'name': 'PAPRIKA - Mediterranean Kitchen & Bar',
  'address': 'Rumunská 8/16, 120 00 Vinohrady, Czechia',
  'place_id': 'ChIJvb0ESYmUC0cRmA9ecAWTok8',
  'price_level': 1,
  'rating': 4.7,
  'no_of_reviews': 1260,
  'location': (50.07383249999999, 14.4321147)},
 {'name': 'Bastion Prague Restaurant',
  'address': 'Horská 1751, 128 00 Nové Město, Czechia',
  'place_id': 'ChIJd7kyj2GUC0cRK8h-gH3M1aQ',
  'price_level': 3,
  'rating': 4.1,
  'no_of_reviews': 233,
  'location': (50.06786229999999, 14.4269231

In [9]:
#Adjusting the list before transforming it into a pandas DataFrame
#Putting the coordinates into a tuple
for restaurant in restaurants_filtered:
    restaurant['location']=tuple([restaurant['location']['lat'],restaurant['location']['lng']])
#Removing opening hours since they do not seem to be informative and we need to decrease the file size
for restaurant in restaurants_filtered:
    restaurant.pop('opening_hours')

In [11]:
#Creating a pandas DataFrame
df=pd.DataFrame(restaurants_filtered)

In [12]:
#Writing into a csv file
df.to_csv('restaurants_google_maps_places_API.csv')

In [14]:
#Trying to read the file
df2=pd.read_csv('restaurants_google_maps_places_API.csv')